<h1 align="center"><font color="yellow">PGVector</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

In [3]:
from dotenv import load_dotenv
#from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from psycopg.conninfo import make_conninfo
import psycopg

import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')


embeddings = OpenAIEmbeddings()

connectionString = make_conninfo(
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    dbname=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD')
)

In [4]:
# Carregar documentos de amostra no banco de dados:

directory = './data'

# Percorra todos os arquivos no diretório, crie incorporações, salve-os no banco de dados:
for file_name in os.listdir(directory):
  file_path = os.path.join(directory, file_name)
  with open(file_path, 'r') as file:
      content = file.read()
      embedding_results = embeddings.embed_query(content)
      with psycopg.connect(connectionString) as conn:
        insert_query = "INSERT INTO documents (file_name, content, embedding) VALUES (%s, %s, %s)"
        insert_data = (file_name, content, embedding_results)
        conn.execute(insert_query, insert_data)
        conn.commit()

# NOTA: Você precisa criar o TABELA, lá no DB postgres, assim:
'''
CREATE TABLE documents (
    id serial PRIMARY KEY,
    file_name varchar,
    content text,
    embedding double precision[]
);
'''        


'\nCREATE TABLE documents (\n    id serial PRIMARY KEY,\n    file_name varchar,\n    content text,\n    embedding double precision[]\n);\n'

In [9]:
#search_text = "Como se chamava a tartaruga?"
search_text = "Qual era o sonho de Leonardo?" #"como se chamava a tartaruga?"

# search do texto Embedding:
embedding_results = embeddings.embed_query(search_text)

print(type(embedding_results))
len(embedding_results)


<class 'list'>


1536

In [10]:
print(embedding_results)

[0.006942690704774008, 0.0060724368279351905, 0.023554866756893347, -0.003980604868068611, -0.005285985683326838, 0.018629876598876917, -0.019145581565163744, -0.0013166616925935366, -0.010056265469044291, -0.00448986461658086, 0.017843424988607286, 0.016670193256638696, 0.02774497822270748, -0.016451017668078105, 0.008470469950310752, 0.0008182802572401579, 0.024586278552112116, -0.018333345358505563, 0.037053468976064964, -0.020344597893520885, 0.020396168948943104, -0.012254461450150675, 0.009256920629257823, 0.005489044734991383, 0.0056824343301795826, 0.009888660935905921, 0.01807549194403959, -0.019957817771821925, 0.022600810613485345, -0.0423910238521689, 0.050822815976574266, 0.01027544012628232, -0.007961209736137223, -0.0015946593811957232, -0.03442336936391838, -0.006053097914982498, 0.014581584138490046, -0.0027590265522102106, 0.002894399408540334, -0.012131981357676106, -0.0020982783417943557, 0.008470469950310752, 0.00028222810491441806, -0.010301225653993428, -0.034268

In [11]:
# Encontre documentos com base na SIMILARIDADE de COSSENO com o texto de pesquisa Embedding:
# with psycopg.connect(connectionString) as conn:
#    select_query = "SELECT * FROM documents"
#    query_data = (f'{embedding_results}', 0.8, 10)
#    results = conn.execute(select_query, query_data).fetchall()
#    for result in results:
#      print(result) 



# Desta maneira não funcionou. Por isso fiz da maneira a seguir:

In [12]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Conectando ao banco de dados
with psycopg.connect(connectionString) as conn:
    cursor = conn.cursor()

    # Selecionar todos os documentos e seus embeddings da tabela
    select_documents_query = "SELECT id, file_name, content, embedding FROM documents;"
    cursor.execute(select_documents_query)
    results = cursor.fetchall()

    similar_documents = []

    for doc_id, doc_file_name, doc_content, doc_embedding in results:
        embedding_doc_DB = np.array(doc_embedding)  # Substitua pelo nome correto da coluna de embeddings
        similarity = cosine_similarity([embedding_results], [embedding_doc_DB])[0][0]

        if similarity >= 0.70:
            similar_documents.append((doc_id, doc_file_name, doc_content, similarity))

    # Ordenar os documentos pela similaridade de cosseno
    similar_documents.sort(key=lambda x: x[3], reverse=True)

    # Imprimir os documentos e suas similaridades
    for doc_id, doc_file_name, doc_content, similarity in similar_documents:
        print(f"Similaridade: {similarity:.4f}")
        print(f"Documento ID: {doc_id}")
        print("Nome do arquivo:", doc_file_name)
        print("Conteúdo:", doc_content)
        print("=" * 50)


Similaridade: 0.8745
Documento ID: 2
Nome do arquivo: tartaruga_espacial.txt
Conteúdo: Era uma vez uma tartaruga chamada Leonardo que sonhava em ir para o espaço. Ele deitava de costas e olhava para as estrelas, imaginando como seria explorar a vastidão do universo.

Um dia, Leonardo estava nadando no oceano quando viu um objeto estranho no céu. Era uma nave espacial! Leonardo nunca tinha visto nada parecido antes. Ele nadou mais perto para ver melhor.

A nave pousou na praia e uma porta se abriu. Um grupo de alienígenas saiu. Eles eram altos e verdes, com olhos grandes e pescoços longos. Leonardo ficou assustado no começo, mas os alienígenas sorriram para ele e o cumprimentaram.

Os alienígenas disseram a Leonardo que eram de um planeta distante. Eles vieram à Terra para aprender sobre seus habitantes. Leonardo ficou feliz em ajudá-los. Ele contou a eles sobre sua vida na Terra e sobre seu sonho de ir para o espaço.

Os alienígenas ficaram impressionados com o conhecimento de Leonardo